### WAP to evaluate the performance of implemented three-layer neural network with variations in activation functions, size of hidden layer, learning rate, batch size and number of epochs. 

#### Description of the Model

* The MNIST dataset is loaded using tensorflow library
* In the model, the architecture is customizable in which different activation functions and hidden layer sizes, providing flexibility for experimentation.
* The activation functions are Sigmoid, Relu, Tanh.
* the hidden layer varies as 256, 128 and 64.
* An input layer with 784 neurons(flattened 28*28 pixels).
* Epoch: 50 is used
* Learning rate: 0.1 is taken
* An output layer of 10 neurons.
* Batch size: 10
* Sigmoid for probabilistic interpretation.
* Tanh for symmetric output between -1 and 1.
* ReLU for sparse activations and faster training.
* Optimizer : SGD(Stochastic Gradient Descent) is used to minimize the loss function by adjusting model parameters using gradients.
* A Confusion Matrix is plotted using Seaborn to visualize classification performance.


#### Description of the code

* Import of libraries
    - TensorFlow for building and training the neural network.
    - NumPy for numerical operations.
    - Matplotlib for plotting graphs.
    - Seaborn for creating a heatmap visualization of the confusion matrix.
    - Sklearn for calculating the confusion matrix.

* Load the dataset
    - The MNIST dataset is loaded using tf.keras.datasets.mnist.load_data().
    - Data is reshaped from 28x28 to 784-dimensional vectors.
    - The pixel values are normalized to the range [0, 1] for faster convergence.

**Network Initialization**:
   - Defines the structure with hidden layers.
   - Initializes weights and biases.
      * W1, W2, b1 and b2 are weights and biases of the hidden layer.
      * W3 and b3 are weights and biases of the output layer.
      * Weights are initialized randomly between -1 and 1. 

**Forward pass**
* The input is passed through the network with: Z1 = XW1+b1, A1= activation(Z1), Z2 = A1W2 + b2
* The output logits are produced without applying softmax. Softmax is applied later for loss computation.
* The activation function (e.g., Sigmoid,ReLU or Tanh) is applied to the hidden layer.

**Loss Function**
* The loss function used is Softmax Cross-Entropy. It measures the difference between predicted logits and actual labels.
* Stochastic Gradient Descent (SGD) is used to minimize the loss by adjusting the model parameters.

**Training Model**
* Gradient Tape:It allows you to compute gradients of a loss function with respect to model parameters like weights and biases.
* The model trains for 50 epochs, computing the average loss for each epoch using mini-batches.
* The loss value is stored for visualization later.
* Execution time is calculated over the training loop.
* After each epoch, predictions are made using the training data.
* Accuracy is calculated by comparing predictions to actual labels.

Loss Curve, Confusion matrix and accuracy curve is plotted.








#### Performance Evaluation

* Performance has been evaluated by plotting Loss curve , Training Accuracy
curve and Confusion matrix.
* The test accuracy for Network architecture 1 is Test Accuracy: 98.35% as it contains a single hidden layer 256 and
activation function is ReLU , it has executed in least time(2990.97seconds).
* The test accuracy for Network architecture has 9 configurations vary according to the model parameters.
* The test accoracy is maximum for the hidden layer 256. As the size of the hidden layer decreases accuracy also decreases.
* The test accuracy varies between 97-98% for the activation function.
* The maximum exection time taken is by Tanh activation function(hidden layer 256) is *5564.17sec* and the minimum time taken is by sigmoid activation function (hidden layer 64) is *2617.29sec*.

#### My comments(Limitations and Scope for improvemnet)

* Maximum test accuracy reached are Test Accuracy: 98.35% for configuration of hidden layer 256 and activation function is ReLU.
* To improve accuracy and decrease execution time , a different activation
function can be used , batch size can be increased , the learning rate value can
be changed , the number of hidden layers and number of hidden layer neurons
need to be changed .